<a href="https://colab.research.google.com/github/lustraka/Data_Analysis_Workouts/blob/main/Analyse_Twitter_Data/wrangle_act.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Wrangling and Analyze Data

In [1]:
# Import dependencies
import requests
import os
import json
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [2]:
path_csv = 'https://raw.githubusercontent.com/lustraka/Data_Analysis_Workouts/main/Analyse_Twitter_Data/'
dfa = pd.read_csv(path_csv+'twitter-archive-enhanced.csv')
dfa.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [3]:
url_tsv = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url_tsv)
with open('image-predictions.tsv', 'wb') as file:
  file.write(r.content)
dfi = pd.read_csv('image-predictions.tsv', sep='\t')
dfi.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [4]:
consumer_key = 'hidden'
consumer_secret = 'hidden'
access_token = 'hidden'
access_secret = 'hidden'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
from timeit import default_timer as timer

count = 0
fails_dict =  {}
start = timer()

if 'tweet_json.txt' in os.listdir():
  os.remove('tweet_json.txt')

with open('tweet_json.txt', 'a') as file:
  for tweet_id in dfa.tweet_id.values:
    count += 1
    if count % 42 == 0:
      print(str(count) + ' (' + str(tweet_id), end=') :')
    try:
      status = api.get_status(tweet_id, tweet_mode='extended')._json
      if count % 42 == 0:
        print("Success")
      file.write(json.dumps(status, ensure_ascii=False)+'\n')
    except tweepy.TweepError as e:
      if count % 42 == 0:
        print('Fail')
      fails_dict[tweet_id] = e
      pass
    except e:
      print('Fail', e)
end = timer()
print(f'Elapsed time: {end - start}')
print(fails_dict)

42 (884441805382717440) :Success
84 (876537666061221889) :Success
126 (868622495443632128) :Success
168 (859851578198683649) :Success
210 (852226086759018497) :Success
252 (844979544864018432) :Success
294 (837820167694528512) :Success
336 (832645525019123713) :Success
378 (828011680017821696) :Success
420 (822244816520155136) :Success
462 (817536400337801217) :Success
504 (813066809284972545) :Success
546 (805826884734976000) :Success
588 (799757965289017345) :Success
630 (794355576146903043) :Success
672 (789960241177853952) :Success
714 (784183165795655680) :Success
756 (778748913645780993) :Success
798 (773191612633579521) :Success
840 (767191397493538821) :Success
882 (760521673607086080) :Success


Rate limit reached. Sleeping for: 803


924 (755955933503782912) :Success
966 (750429297815552001) :Success
1008 (747600769478692864) :Success
1050 (743222593470234624) :Success
1092 (737445876994609152) :Success
1134 (728751179681943552) :Success
1176 (720043174954147842) :Success
1218 (714962719905021952) :Success
1260 (710272297844797440) :Success
1302 (707629649552134146) :Success
1344 (704761120771465216) :Success
1386 (700747788515020802) :Success
1428 (697630435728322560) :Success
1470 (694183373896572928) :Success
1512 (691416866452082688) :Success
1554 (688908934925697024) :Success
1596 (686358356425093120) :Success
1638 (684195085588783105) :Success
1680 (682032003584274432) :Success
1722 (680130881361686529) :Success
1764 (678446151570427904) :Success


Rate limit reached. Sleeping for: 804


1806 (676942428000112642) :Success
1848 (675820929667219457) :Success
1890 (674774481756377088) :Success
1932 (674036086168010753) :Success
1974 (673213039743795200) :Success
2016 (672205392827572224) :Success
2058 (671355857343524864) :Success
2100 (670717338665226240) :Success
2142 (669993076832759809) :Success
2184 (668989615043424256) :Success
2226 (668286279830867968) :Success
2268 (667524857454854144) :Success
2310 (666804364988780544) :Success
2352 (666049248165822465) :Success
Elapsed time: 1870.482612943
{888202515573088257: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 873697596434513921: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 872668790621863937: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 872261713294495745: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 869988702071779329: TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 866816280283807

Data gathered form Twitter API:

| Attribute | Type | Description |
| --- | :-: | --- |
| id | int | The integer representation of unique identifier for this Tweet |
| retweet_count | int | Number of times this Tweet has been retweeted. |
| favorite_count | int | *Nullable*. Indicates approximately how many times this tweet has been liked by Twitter users. |

Reference: [Tweepy docs: Tweet Object](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/tweet)



In [6]:
df_tweets = []
with open('tweet_json.txt', 'r') as file:
  line = file.readline()
  while line:
    status = json.loads(line)
    df_tweets.append({'tweet_id': status['id'], 'retweet_count': status['retweet_count'], 'favorite_count': status['favorite_count']})
    line = file.readline()
dft = pd.DataFrame(df_tweets)
dft.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7160,34447
1,892177421306343426,5385,29872
2,891815181378084864,3550,22486
3,891689557279858688,7375,37659
4,891327558926688256,7918,35974


## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



### The archive `twitter_archive_enhanced.csv` (alias `dba`)
> "I extracted this data programmatically, but I didn't do a very good job. The ratings probably aren't all correct. Same goes for the dog names and probably dog stages (see below for more information on these) too. You'll need to assess and clean these columns if you want to use them for analysis and visualization."

In [7]:
dfa.sample(15)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
208,852311364735569921,NaN,NaN,2017-04-13 00:03:59 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Wiggles. She would like you to spot he...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/852311364...,13,10,Wiggles,None,None,None,None
1980,672975131468300288,NaN,NaN,2015-12-05 03:05:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Chuckles. He is one skeptical pupper. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/672975131...,10,10,Chuckles,None,None,pupper,None
1085,738184450748633089,NaN,NaN,2016-06-02 01:44:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...","For the last time, we only rate dogs. Pls stop...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/738184450...,9,10,None,None,None,None,None
2353,666033412701032449,NaN,NaN,2015-11-15 23:21:54 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a very happy pup. Big fan of well-main...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/666033412...,9,10,a,None,None,None,None
702,786036967502913536,NaN,NaN,2016-10-12 02:53:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Scout. He really wants ...,7.798343e+17,4.196984e+09,2016-09-25 00:06:08 +0000,https://twitter.com/dog_rates/status/779834332...,11,10,Scout,None,None,None,None
596,798697898615730177,NaN,NaN,2016-11-16 01:23:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Stubert. He just arrive...,7.128090e+17,4.196984e+09,2016-03-24 01:11:29 +0000,https://twitter.com/dog_rates/status/712809025...,10,10,Stubert,None,None,None,None
2063,671159727754231808,NaN,NaN,2015-11-30 02:52:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Anthony. He just finished up his maste...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671159727...,5,10,Anthony,None,None,None,None
585,800018252395122689,NaN,NaN,2016-11-19 16:49:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a doggo doin a struggle. 11/10 much det...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/800018252...,11,10,None,doggo,None,None,None
928,754747087846248448,NaN,NaN,2016-07-17 18:38:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Keith. He's pursuing a more 2D lifesty...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/754747087...,12,10,Keith,None,None,None,None
1214,715200624753819648,NaN,NaN,2016-03-30 15:34:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Michelangelope. He's half coffee cup. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/715200624...,12,10,Michelangelope,None,None,None,None


In [8]:
dfa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [9]:
for col in dfa.columns[[10,11,13,14,15,16]]:
  print(dfa[col].unique())

[  13   12   14    5   17   11   10  420  666    6   15  182  960    0
   75    7   84    9   24    8    1   27    3    4  165 1776  204   50
   99   80   45   60   44  143  121   20   26    2  144   88]
[ 10   0  15  70   7  11 150 170  20  50  90  80  40 130 110  16 120   2]
['None' 'doggo']
['None' 'floofer']
['None' 'pupper']
['None' 'puppo']


#### Curated `twitter_archive_enhanced.csv` info

| # | Variable | Non-Null | Nunique | Dtype | Notes |
|---|----------|----------|---------|-------|-------|
| 0 | tweet_id | 2356 | 2356 | int64  | |
| 1 | in_reply_to_status_id | 78 | 77 | float64 | these tweets are replies |
| 2 | in_reply_to_user_id | 78 | 31 | float64 | see $\uparrow$ |
| 3 | timestamp | 2356 | 2356 | object | object $\to$ datetime | 
| 4 | source | 2356 | 4 | object | |
| 5 | text | 2356 | 2356 | object | some tweets don't have an image (1) |
| 6 | retweeted_status_id | 181 | 181 | float64 | these are retweets |
| 7 | retweeted_status_user_id | 181 | 25 | float64 | see $\uparrow$ |
| 8 | retweeted_status_timestamp | 181 | 181 | object | see $\uparrow$ |
| 9 | expanded_urls | 2297 | 2218 | object | missing values |
| 10 | rating_numerator | 2356 | 40 | int64  | entries with numerator $> 20$ may be incorrect (4a) |
| 11 | rating_denominator | 2356 | 18 | int64  | entries with denominator $\neq 10$ may be incorrect (4b) |
| 12 | name | 2356 | 957 | object | incorrect names or missing values (2) |
| 13 | doggo | 2356 | 2 | object | a value as a column + (3) some misclassified stages|
| 14 | floofer | 2356 | 2 | object | see $\uparrow$ |
| 15 | pupper | 2356 | 2 | object | see $\uparrow$ |
| 16 | puppo | 2356 | 2 | object | see $\uparrow$ |

Source: visual and programmatic assessment

```python
# #, Variable, Non-Null (Count), Dtype:
dfa.info()
# Nunique:
dfa.nunique()
# Check unique values
for col in dfa.columns[[10,11,13,14,15,16]]:
  print(dfa[col].unique())
# Notes
# (1) Some tweets don't have an image
dfa.loc[dfa.text.apply(lambda s: 'https://t.co' not in s)].shape[0]
# [Out] 124
```

In [10]:
# (2a) Incorrect names - begin with a lowercase
import re
print(re.findall(r';([a-z].*?);', ';'.join(dfa.name.unique())))

['such', 'a', 'quite', 'not', 'one', 'incredibly', 'mad', 'an', 'very', 'just', 'my', 'his', 'actually', 'getting', 'this', 'all', 'old', 'infuriating', 'the', 'by', 'officially', 'life', 'light', 'space']


In [11]:
# (2b) Incorrect names - None
dfa.loc[dfa.name == 'None'].shape[0]

745

In [12]:
# (3a) Misclassified stages - indicated in the stage but not present in the text
stages = ['doggo', 'pupper', 'puppo', 'floofer']
print('Stage     | Total | Misclassified |')
print('-'*35)
for stage in stages:
  total = dfa.loc[dfa[stage] == stage].shape[0]
  missed = dfa.loc[(dfa[stage] == stage) & (dfa.text.apply(lambda s: stage not in s.lower()))].shape[0]
  print(f"{stage.ljust(9)} | {total:5d} | {missed:13d} |")

Stage     | Total | Misclassified |
-----------------------------------
doggo     |    97 |             0 |
pupper    |   257 |             0 |
puppo     |    30 |             0 |
floofer   |    10 |             0 |


In [13]:
# (3b) Misclassified stages - not indicated in the stage but is present in the text
stages = ['doggo', 'pupper', 'puppo', 'floofer']
print('Stage     | Total | Misclassified |')
print('-'*35)
for stage in stages:
  total = dfa.loc[dfa[stage] == stage].shape[0]
  missed = dfa.loc[(dfa[stage] != stage) & (dfa.text.apply(lambda s: stage in s.lower()))].shape[0]
  print(f"{stage.ljust(9)} | {total:5d} | {missed:13d} |")

Stage     | Total | Misclassified |
-----------------------------------
doggo     |    97 |            10 |
pupper    |   257 |            26 |
puppo     |    30 |             8 |
floofer   |    10 |             0 |


##### Note (4) Ratings where `rating_numerator` $ > 20$ or  `rating_denomiator` $\neq 10$
Code used:
```python
# Show the whole text
pd.options.display.max_colwidth = None

# (4a) Show tweets with possibly incorrect rating : rating_numerator > 20
dfa.loc[dfa.rating_numerator > 20, ['text', 'rating_numerator', 'rating_denominator']]

# (4b) Show tweets with possibly incorrect rating : rating_denominator != 10
dfa.loc[dfa.rating_denominator != 10, ['text', 'rating_numerator', 'rating_denominator']]

```
In cases where users used float numbers, such as 9.75/10 or 11.27/10, we will use the floor rounding, i.e. 9/10 or 11/10 respectively. We will correct only those rating which were incorrectly identified in the text. Ratings with weird values used in the text are left unchanged cos they're good dogs Brent.

Results:

In [14]:
# Show the whole text
pd.options.display.max_colwidth = None

# Fill dict with key = index and value = correct rating
incorrect_rating = {313 : '13/10', 340 : '9/10', 763: '11/10', 313 : '13/10', 784 : '14/10', 1165 : '13/10', 1202 : '11/10', 1662 : '10/10', 2335 : '9/10'}

# Indicate tweets with missing rating
missing_rating = [342, 516]

# Show tweet with incorrectly identified rating
dfa.loc[list(incorrect_rating.keys()), ['text', 'rating_numerator', 'rating_denominator']]

,text,rating_numerator,rating_denominator
313,"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",960,0
340,"RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wu…",75,10
763,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,27,10
784,"RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…",9,11
1165,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4,20
1202,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50,50
1662,This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5,7,11
2335,This is an Albanian 3 1/2 legged Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv,1,2


### The Tweet Image Predictions `image_predictions.tsv`
>A table full of image predictions (the top three only) alongside each tweet ID, image URL, and the image number that corresponded to the most confident prediction (numbered 1 to 4 since tweets can have up to four images)

In [15]:
dfi.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
868,697596423848730625,https://pbs.twimg.com/media/Ca5cPrJXIAImHtD.jpg,1,Shetland_sheepdog,0.621668,True,collie,0.366578,True,Pembroke,0.007698,True
1432,773308824254029826,https://pbs.twimg.com/media/CrtYRMEWIAAUkCl.jpg,1,shopping_cart,0.572349,False,Labrador_retriever,0.151406,True,shopping_basket,0.107102,False
41,666701168228331520,https://pbs.twimg.com/media/CUCZLHlUAAAeAig.jpg,1,Labrador_retriever,0.887707,True,Chihuahua,0.029307,True,French_bulldog,0.020756,True
1883,847251039262605312,https://pbs.twimg.com/media/C8IKUjAUwAEP-En.jpg,1,Airedale,0.495380,True,Irish_terrier,0.316456,True,Lakeland_terrier,0.158533,True
1043,712668654853337088,https://pbs.twimg.com/media/CePoVTyWsAQEz1g.jpg,1,Labrador_retriever,0.829058,True,golden_retriever,0.038664,True,Chihuahua,0.026221,True
1453,776819012571455488,https://pbs.twimg.com/media/CW88XN4WsAAlo8r.jpg,3,Chihuahua,0.346545,True,dalmatian,0.166246,True,toy_terrier,0.117502,True
886,699036661657767936,https://pbs.twimg.com/media/CbN6IW4UYAAyVDA.jpg,1,Chihuahua,0.222943,True,toyshop,0.179938,False,Weimaraner,0.163033,True
957,705442520700944385,https://pbs.twimg.com/media/Cco8OmOXIAE0aCu.jpg,1,Great_Pyrenees,0.309106,True,kuvasz,0.224556,True,seat_belt,0.202100,False
659,682393905736888321,https://pbs.twimg.com/media/CXhZom1UwAA4Zz6.jpg,1,vizsla,0.657275,True,paddle,0.090286,False,Rhodesian_ridgeback,0.048228,True
1971,869227993411051520,https://pbs.twimg.com/media/DBAePiVXcAAqHSR.jpg,1,Pembroke,0.664181,True,Chihuahua,0.169234,True,Cardigan,0.132700,True


In [16]:
dfi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


#### Curated Info

| # | Variable | Non-Null | Nunique | Dtype | Notes |
|---|----------|----------|---------|-------|-------|
| 0 | tweet_id | 2075 | 2078 | int64 | |
| 1 | jpg_url | 2075 | 2009 | object | |
| 2 | img_num | 2075 | 4 | int64 | the image number that corresponded to the most confident prediction|
| 3 | p1 | 2075 | 378 | object | prediction |
| 4 | p1_conf | 2075 | 2006 | float64 | confidence of prediction |
| 5 | p1_dog | 2075 | 2 | int64 | Is the prediction a breed of dog? : int $\to$ bool |
| 6 | p2 | 2075 | 405 | object | dtto |
| 7 | p2_conf | 2075 | 2004 | float64 | dtto |
| 8 | p2_dog | 2075 | 2 | int64 | dtto |
| 9 | p3 | 2075 | 408 | object | dtto |
| 10 | p3_conf | 2075 | 2006 | float64 | dtto |
| 11 | p3_dog | 2075 | 2 | int64 | dtto |

Source: visual and programmatic assessment

```python
# #, Variable, Non-Null (Count), Dtype:
dfa.info()
# Nunique:
dfa.nunique()
```


### Additional Data From Twitter API

In [17]:
dft.sample(10)

,tweet_id,retweet_count,favorite_count
1558,686760001961103360,1295,3312
1049,739544079319588864,19723,37207
556,801115127852503040,2014,7828
476,813202720496779264,1741,8944
158,860177593139703809,27403,0
883,757725642876129280,1130,4330
1968,672591271085670400,169,656
1174,716439118184652801,197,2251
1215,711968124745228288,2143,7592
621,793241302385262592,3159,10189


In [18]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2327 entries, 0 to 2326
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   tweet_id        2327 non-null   int64
 1   retweet_count   2327 non-null   int64
 2   favorite_count  2327 non-null   int64
dtypes: int64(3)
memory usage: 54.7 KB


#### Curated Info

| # | Variable | Non-Null | Nunique | Dtype | Notes |
|---|----------|----------|---------|-------|-------|
| 0 | tweet_id | 2327 | 2327 | int64 | |
| 1 | retweet_count | 2327 | 1671 | int64 | |
| 2 | favorite_count | 2327 | 2006 | int64 | |

Source: visual and programmatic assessment

```python
# #, Variable, Non-Null (Count), Dtype:
dfa.info()
# Nunique:
dfa.nunique()
```

### Quality issues
1. Replies are not original tweets.
2. Retweets are not original tweets.
3. Some tweets don't have any image
4. Some ratings are incorrectly identified
5. Some ratings are missing
6. Names starting with lowercase are incorrect
7. Names with value None are incorrect
8. Column timestamp has the dtype object (string)


### Tidiness issues
1. Dogs' stages (doggo, pupper, puppo, floofer) as columns
2. Multiple image predictions in one row
3. Data in multiple datasets

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [19]:
# Make copies of original pieces of data
dfa_clean = dfa.copy() # archive
dfi_clean = dfi.copy() # image predictions
dft_clean = dft.copy() # data from Twitter API

### Q1: Replies are not original tweets.

#### Define:
Remove replies from `dfa_clean` dataframe and drop variables *in_reply_to_status_id* and *in_reply_to_user_id*.

#### Code

In [20]:
dfa_clean = dfa_clean.loc[dfa_clean.in_reply_to_status_id.isna()]
print(dfa_clean.in_reply_to_status_id.notna().sum())
dfa_clean.drop(columns=['in_reply_to_status_id', 'in_reply_to_user_id'], inplace=True)

0


#### Test

In [21]:
dfa_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2355
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2278 non-null   int64  
 1   timestamp                   2278 non-null   object 
 2   source                      2278 non-null   object 
 3   text                        2278 non-null   object 
 4   retweeted_status_id         181 non-null    float64
 5   retweeted_status_user_id    181 non-null    float64
 6   retweeted_status_timestamp  181 non-null    object 
 7   expanded_urls               2274 non-null   object 
 8   rating_numerator            2278 non-null   int64  
 9   rating_denominator          2278 non-null   int64  
 10  name                        2278 non-null   object 
 11  doggo                       2278 non-null   object 
 12  floofer                     2278 non-null   object 
 13  pupper                      2278 

### Q2: Retweets are not original tweets.

#### Define
Remove retweets from `dfa_clean` and drop variables *retweeted_status_id*, *retweeted_status_user_id*, and *retweeted_status_timestamp*.

#### Code

In [22]:
dfa_clean = dfa_clean.loc[dfa_clean.retweeted_status_id.isna()]
print(dfa_clean.retweeted_status_id.notna().sum())
dfa_clean.drop(columns=['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], inplace=True)

0


#### Test

In [23]:
dfa_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            2097 non-null   int64 
 1   timestamp           2097 non-null   object
 2   source              2097 non-null   object
 3   text                2097 non-null   object
 4   expanded_urls       2094 non-null   object
 5   rating_numerator    2097 non-null   int64 
 6   rating_denominator  2097 non-null   int64 
 7   name                2097 non-null   object
 8   doggo               2097 non-null   object
 9   floofer             2097 non-null   object
 10  pupper              2097 non-null   object
 11  puppo               2097 non-null   object
dtypes: int64(3), object(9)
memory usage: 213.0+ KB


### Q3: Some tweets don't have any image

#### Define
Remove tweets that don't have image from `dfa_clean`.

#### Code

In [24]:
dfa_clean = dfa_clean.loc[dfa_clean.text.apply(lambda s: 'https://t.co' in s)]

#### Test

In [25]:
dfa_clean.loc[dfa_clean.text.apply(lambda s: 'https://t.co' not in s)].shape[0]

0

### Q4: Some ratings are incorrectly identified

#### Define
Updata the incorrect ratings with the correct ones (both stored in a dictionary *incorrect_rating*).

#### Code

In [26]:
# Some observations could have been removed
ratings_to_update = dfa_clean.index.intersection(list(incorrect_rating.keys()))

In [27]:
for rating in ratings_to_update:
  dfa_clean.at[rating,'rating_numerator'] = incorrect_rating[rating].split('/')[0]
  dfa_clean.at[rating, 'rating_denominator'] = incorrect_rating[rating].split('/')[1]

#### Test

In [28]:
# Show the whole text
pd.options.display.max_colwidth = None
# Show tweets
dfa_clean.loc[ratings_to_update, ['text', 'rating_numerator', 'rating_denominator']]

,text,rating_numerator,rating_denominator
763,This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back https://t.co/QFaUiIHxHq,11,10
1165,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,13,10
1202,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,11,10
1662,This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5,10,10
2335,This is an Albanian 3 1/2 legged Episcopalian. Loves well-polished hardwood flooring. Penis on the collar. 9/10 https://t.co/d9NcXFKwLv,9,10


In [29]:
dfa_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            2094 non-null   int64 
 1   timestamp           2094 non-null   object
 2   source              2094 non-null   object
 3   text                2094 non-null   object
 4   expanded_urls       2094 non-null   object
 5   rating_numerator    2094 non-null   int64 
 6   rating_denominator  2094 non-null   int64 
 7   name                2094 non-null   object
 8   doggo               2094 non-null   object
 9   floofer             2094 non-null   object
 10  pupper              2094 non-null   object
 11  puppo               2094 non-null   object
dtypes: int64(3), object(9)
memory usage: 292.7+ KB


### Q5: Some ratings are missing

#### Define
Delete observations with missing rating in `dfa_clean` identified in the variable *missing_rating*.

#### Code

In [30]:
# Some observations could have been removed
tweets_to_delete = dfa_clean.index.intersection(missing_rating)
# Delete tweets without rating
dfa_clean.drop(index=tweets_to_delete, inplace=True)

#### Test

In [31]:
# Should be empty
dfa_clean.index.intersection(missing_rating)

Int64Index([], dtype='int64')

### Q6: Names starting with lowercase are incorrect

#### Define
Replace names starting with a lowercase letter in `dfa_clean` with empty string.

#### Code

In [32]:
incorrect_names = re.findall(r';([a-z].*?);', ';;'.join(dfa_clean.name.unique()))
def clean_names(name):
  """If a name is in a global variable `incorrect_names`, 
      replace it by empty string,
      otherwise return the original name."""

  if name in incorrect_names:
    return ''
  else:
    return name

dfa_clean['name'] = dfa_clean.name.apply(clean_names)

#### Test

In [33]:
# Should be empty
print(re.findall(r';([a-z].*?);', ';;'.join(dfa_clean.name.unique())))

[]


### Q7: Names with value None are incorrect

#### Define
Replace names 'None' in `dfa_clean` with an empty string.

#### Code

In [34]:
dfa_clean['name'] = dfa_clean.name.apply(lambda name: '' if name == 'None' else name)

#### Test

In [35]:
# Should be zero
dfa_clean.query('name == "None"').shape[0]

0

In [36]:
dfa_clean.name.value_counts()[:10]

           704
Lucy        11
Charlie     11
Cooper      10
Oliver      10
Tucker       9
Penny        9
Lola         8
Sadie        8
Winston      8
Name: name, dtype: int64

### Q8: Column timestamp has the dtype object (string)

#### Define
Convert variable *timestamp* in `dfa_clean` to datetime.

#### Code

In [37]:
dfa_clean['timestamp'] = pd.to_datetime(dfa_clean.timestamp)

#### Test

In [38]:
dfa_clean.timestamp.dtype

datetime64[ns, UTC]

### T1: Dogs' stages (doggo, pupper, puppo, floofer) as columns

#### Define
Derive a new variable *stage* from variables *doggo, pupper, puppo, floofer*. Fill an empty string if no stage indicated. Then drop exploited variables.

#### Code

In [39]:
def get_stage(row):
  """Fill the stage or an empty string (if the stage is not identified)."""

  stage = set([row['doggo'], row['pupper'], row['puppo'], row['floofer']])
  if len(stage) > 1:
    return list(stage.difference({'None'}))[0]
  else:
    return ''

dfa_clean['stage'] = dfa_clean.apply(get_stage, axis=1)
dfa_clean.drop(columns=['doggo', 'pupper', 'puppo', 'floofer'], inplace=True)

#### Test

In [40]:
dfa_clean.stage.value_counts()

           1758
pupper      229
doggo        74
puppo        23
floofer       9
Name: stage, dtype: int64

In [41]:
dfa_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2093 entries, 0 to 2355
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            2093 non-null   int64              
 1   timestamp           2093 non-null   datetime64[ns, UTC]
 2   source              2093 non-null   object             
 3   text                2093 non-null   object             
 4   expanded_urls       2093 non-null   object             
 5   rating_numerator    2093 non-null   int64              
 6   rating_denominator  2093 non-null   int64              
 7   name                2093 non-null   object             
 8   stage               2093 non-null   object             
dtypes: datetime64[ns, UTC](1), int64(3), object(5)
memory usage: 163.5+ KB


### T2: Multiple image predictions in one row

#### Define
Extract the most confident prediction of a breed of dog. Drop exploited columns and remove observation without a prediction.

#### Code

In [42]:
def get_breed(row):
  """Extract the most confident prediction of a breed of dog."""

  predictions = [[row['p1'], row['p1_conf'], row['p1_dog']],
                 [row['p2'], row['p2_conf'], row['p2_dog']],
                 [row['p3'], row['p3_conf'], row['p3_dog']]]
  
  # Filter predictions of a bread of dog
  dogs = list(filter(lambda x: x[2], predictions))
  # Sort predictions accoring to confidence
  best = sorted(dogs, key=lambda x: x[1], reverse=True)

  # Return the best prediction
  if len(best) == 0:
    return ''
  else:
    return str(best[0][0]).replace('_', ' ').title()

dfi_clean['breed'] = dfi_clean.apply(get_breed, axis=1)
dfi_clean.drop(columns=['p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], inplace=True)

In [43]:
# Remove tweets without a prediction
dfi_clean = dfi_clean.query('breed != ""')

#### Test

In [44]:
dfi_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1751 entries, 0 to 2073
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tweet_id  1751 non-null   int64 
 1   jpg_url   1751 non-null   object
 2   img_num   1751 non-null   int64 
 3   breed     1751 non-null   object
dtypes: int64(2), object(2)
memory usage: 68.4+ KB


In [45]:
print(sorted(dfi_clean.breed.unique()))

['Afghan Hound', 'Airedale', 'American Staffordshire Terrier', 'Appenzeller', 'Australian Terrier', 'Basenji', 'Basset', 'Beagle', 'Bedlington Terrier', 'Bernese Mountain Dog', 'Black-And-Tan Coonhound', 'Blenheim Spaniel', 'Bloodhound', 'Bluetick', 'Border Collie', 'Border Terrier', 'Borzoi', 'Boston Bull', 'Bouvier Des Flandres', 'Boxer', 'Brabancon Griffon', 'Briard', 'Brittany Spaniel', 'Bull Mastiff', 'Cairn', 'Cardigan', 'Chesapeake Bay Retriever', 'Chihuahua', 'Chow', 'Clumber', 'Cocker Spaniel', 'Collie', 'Curly-Coated Retriever', 'Dalmatian', 'Dandie Dinmont', 'Doberman', 'English Setter', 'English Springer', 'Entlebucher', 'Eskimo Dog', 'Flat-Coated Retriever', 'French Bulldog', 'German Shepherd', 'German Short-Haired Pointer', 'Giant Schnauzer', 'Golden Retriever', 'Gordon Setter', 'Great Dane', 'Great Pyrenees', 'Greater Swiss Mountain Dog', 'Groenendael', 'Ibizan Hound', 'Irish Setter', 'Irish Terrier', 'Irish Water Spaniel', 'Irish Wolfhound', 'Italian Greyhound', 'Japane

### T3: Data in multiple datasets

#### Define
Merge archive `dfa_clean`, breed predictions `dfi_clean`, and metrics `dft_clean` into `df_clean` for further analysis and visualization

#### Code

In [46]:
df_clean = dfa_clean.merge(dfi_clean, how='inner', on='tweet_id')
df_clean = df_clean.merge(dft_clean, how='inner', on='tweet_id')

#### Test

In [47]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1657 entries, 0 to 1656
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   tweet_id            1657 non-null   int64              
 1   timestamp           1657 non-null   datetime64[ns, UTC]
 2   source              1657 non-null   object             
 3   text                1657 non-null   object             
 4   expanded_urls       1657 non-null   object             
 5   rating_numerator    1657 non-null   int64              
 6   rating_denominator  1657 non-null   int64              
 7   name                1657 non-null   object             
 8   stage               1657 non-null   object             
 9   jpg_url             1657 non-null   object             
 10  img_num             1657 non-null   int64              
 11  breed               1657 non-null   object             
 12  retweet_count       1657 non-null 

In [48]:
df_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,stage,jpg_url,img_num,breed,retweet_count,favorite_count
0,892177421306343426,2017-08-01 00:17:27+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1,Chihuahua,5385,29872
1,891815181378084864,2017-07-31 00:18:03+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1,Chihuahua,3550,22486
2,891689557279858688,2017-07-30 15:58:51+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1,Labrador Retriever,7375,37659
3,891327558926688256,2017-07-29 16:00:24+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f","https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2,Basset,7918,35974
4,891087950875897856,2017-07-29 00:08:17+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek https://t.co/kQ04fDDRmh,https://twitter.com/dog_rates/status/891087950875897856/photo/1,13,10,,,https://pbs.twimg.com/media/DF3HwyEWsAABqE6.jpg,1,Chesapeake Bay Retriever,2664,18146


## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [49]:
with open('twitter_archive_master.csv', 'w') as file:
  df_clean.to_csv(file)

In [50]:
# Store the dataframe for further processing
from sqlalchemy import create_engine

# Create SQLAlchemy engine and empty database
engine = create_engine('sqlite:///weratedogsdata_clean.db')

# Store dataframes in database
df_clean.to_sql('df_clean', engine, index=False)

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization